In [16]:
import os
os.chdir(os.path.dirname(os.path.abspath("__file__")))

# Introduction to the model
The transmission model is an individual-based model that incorporates demographic and transmission dynamics. Each serotype is represented individually. We assume a susceptible-infectious-susceptible (SIS) model for S. pneumoniae infection, where “infection” represents carriage of S. pneumoniae, and disease outcomes are considered for the infected individuals. Similar to previous modelling studies, the infectious duration for an individual is sampled from an exponential distribution with an age-specific mean infectious duration. Upon recovery, infectious individuals re-enter the susceptible class, with protection derived from antibodies according to the time since their last vaccination. 


### Installing the module
The module installation is described in [README.md](../../README.md) file. The module must be install through terminal:
```shell
python3 -m venv venv
source venv/bin/activate
# set working directory as code repository then
pip install -e .
```

### Running simulations

Given a parameter set, `run_....py` documents in `run_...` folders, takes the code in [src](../src) folder and simulates the transmission. Folder [run_scenarios](../run_scenarios) includes scripts that simulates the transmission in non-Indigenous population of Australia where the population is segregated as _not-at-risk_, _Tier 1 at-risk_ and _Tier 2 at-risk_ individuals. 

#### Running single simulation
In order to run a single simluation, make sure that `run_....py` script has the following lines at the end:

```python
#run a single simulation among combinations
new_go_single(job_inputs[0][0])
#OR run multiple simulations
#results = parq.run(new_go_single, job_inputs, n_proc=32, results=False)
```
#### Running batch simulations
In order to run a batch of simulations using multiprocessing, given for instance 32 cores, make sure that `run_....py` script has the following lines at the end:

```python
#run a single simulation among combinations
#new_go_single(job_inputs[0][0])
#run multiple simulations
results = parq.run(new_go_single, job_inputs, n_proc=32, results=False)
```

Scripts  `run_....py` takes  `new_go_single` function from `...disease_model.py` script. `new_go_single()` function runs `go_single()` function from corresponding `src/model/disease/disease...._run.py` script and it runs simulation from a single parameter set. `new_go_single()` takes inputs as `DiseaseModel` class (which is created in the same script, a contact matrix, and a parameter set). `DiseaseModel` class takes observers from [observers](../src/observers) folder to collect summary statistics. In order to add remove observers the changes musy be made to `DiseaseModel` class in `...disease_model.py` script. In order to create more observers or edit existing observers, changes must be made in the [observers](../src/observers) folder.

Parameter set are taken from `..._params.py` from the same folder and can be further edited in `run_....py` script.

### Demography
The model includes births, ageing, age-specific mortality, and age-specific migration to capture the age distribution of a population over time. It takes inputs of initial population size, initial age distribution, age-specific death rates, annual net birth rates, annual migration rates, and age distribution in the emigrated population to simulate demographic dynamics. We sourced the inputs from Australian Bureau of Statistics and Macro Trends Global to align with Australian population dynamics. 

[class AtRiskDisPopulation(DisPopulation)](../src/model/population/at_risk_varying_disease_population.py) is used as a base population class which adds atrisk column to the population table usin`add_at_risk_column()` from [disease_utils.py](../src/model/disease/disease_utils.py). The base population class is located in [population.py](../src/model/population/population.py) which does not include any disease or vaccine related columns. 

`class AtRiskDisPopulation(DisPopulation)` and `class DisPopulation(Population)` can upload population files stored in [saved_checkpoints](../../data/saved_checkpoints) Folder.

##### Functions of _AtRiskDisPopulation Class_
*TODO*

### Transmission model
### TODO: Update the equations and symbols throughout whole file

##### Mixing assumptions
Transmission within and between age groups is driven by contact rates, derived from synthetic contact matrices . In this work, we have assumed contacts in our population according to the contact matrices provided by Prem et al.

##### Carriage assumptions
We assume that individuals are exposed to a single serotype at every time step. The exposed serotype is randomly selected from weighted serotype distribution in the population:

Pr(Exposure to serotype a)=  ((τ_a  * I_a))/( ∑_s▒〖(τ_s  * I_s )   〗)

where τ_a is the transmission coefficient multiplier of serotype a, I_a is the number of individuals infected by serotype a, and the denominator represents the sum of exposures by all the circulation serotypes, s. We then calculate the transmission of the exposed S. pneumoniae serotype for each individual. Individual infection risk for a particular serotype is determined by serotype prevalence, age, contact rates, current infection status and current antibody levels:

Pr(Infection by serotype a | exposure to serotype a)= (1 - e^(-foi_i )) * V_a  * C

where, foi_i represents the force of infection on an individual in age class i, V_a represents the vaccine-induced probability of acquisition multiplier to serotype a (see Section 2.1), and C represents the reduction of susceptibility given existing infections. We allow co-infections in the model assuming that individuals may be infected with multiple serotypes concurrently, with a reduced probability of acquiring another serotype if already infected, representing within-host competition. In this work, we assumed that C takes values of 1 (fully susceptible), 0.8 and 0 for the existing number of infections of 0, 1, and 2, respectively, which means that individuals could only be infected with two serotypes concurrently.
 The force of infection comprises age-specific community transmission:

〖foi〗_i =q*∑_s▒τ_s  *(∑_j▒〖η_ij   I_js/(N_j  )〗)  

where:

    q  = Transmission coefficient

    τ_s = Transmission coefficient multiplier of serotype s

    η_ij  = Number of daily community contacts between an individual in age group i and individuals in age group j

    I_js = Number of individuals in age group j infected with serotype s
    
    N_j = Number of individuals in age group j




```python
def check_exposure():

def check_recoveries():
```

In addition to community-level circulation, we introduce serotypes externally via two processes.  First, an external exposure rate simulates the ongoing introduction of serotypes through population migration, assumed to have the same age-specific and serotype-specific incidence of S. pneumoniae carriage as the local population (which maintains the same level of serotype and age specific carriage while population size increases). Second, introduction of any serotype sampled uniformly from the set of all possible serotypes and therefore may not be currently circulating within the local population (which allows possibility of serotype replacement under immune selection over time).

### Clinical model

The clinical model takes information on the age and vaccine history of newly infected individuals from the transmission model, and the antibody levels at the time of infection, derived from these values, from the immunity model. The model then determines whether an infected individual develops a disease outcome (IPD or CAP), according to the relative probability of each occurring, and records the disease outcomes when they occur. In this work, we assume that 23vPPV is not effective against acquisition and developing CAP. 

```python
def check_disease():
```

### Vaccine Rollout
The model simulates the implementation of historical vaccine schedules with their rollout years, eligible age range, annual fraction of on-time and late vaccine coverages of the eligible age group, number of doses and time interval between doses. We assume that individuals receiving the first dose on time would also receive the following doses on time. We also assume that maximum protection by a vaccine starts immediately.




```python
def check_vaccines():
```